In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv('/kaggle/input/engineering-placements-prediction/collegePlace.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

STREAMS

In [ ]:
df['Stream'].value_counts().plot(kind='pie')

In [ ]:
print(np.sum(df.isnull()))

In [ ]:
df['Gender'].value_counts()

In [ ]:
data_M=df[df['Gender']=='Male']
data_F=df[df['Gender']=='Female']
print(data_M.shape,data_F.shape)


In [ ]:
male_placement_passed=data_M[data_M['PlacedOrNot']== 1].shape[0]
female_placement_passed=data_F[data_F['PlacedOrNot']==1 ].shape[0]
print(male_placement_passed,female_placement_passed)


In [ ]:
male_placed_precentage=male_placement_passed/data_M.shape[0]
female_placed_precentage=female_placement_passed/data_F.shape[0]



In [ ]:
Information= {

    'Total Male students': data_M.shape[0],
    'Total Female Students': data_F.shape[0],
    '% of male student Placed': np.round(male_placed_precentage*100),
    '% of female student placed':np.round(female_placed_precentage*100)
}

In [ ]:
Information

**LOW CGPA VS PLACEMENTS***** 

In [ ]:
low_cgpa=df[df['CGPA']<df['CGPA'].mean()]

In [ ]:
low_cgpa.shape[0]

In [ ]:
low_cgpa_placed=low_cgpa[low_cgpa['PlacedOrNot']==1]
low_cgpa_placed.shape[0]

In [ ]:
percentage_low_cgpa_placed=(low_cgpa_placed.shape[0]*100)/low_cgpa.shape[0]
print(np.round(percentage_low_cgpa_placed))

In [ ]:
import plotly.express as px
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

In [ ]:
fig=px.histogram(data_frame=df,x='Stream',color='PlacedOrNot',template='plotly_dark')
fig.show()

In [ ]:
fig=px.histogram(data_frame=low_cgpa,x='CGPA',color='PlacedOrNot',barmode='group')
fig.update_layout(bargap=0.2)
fig.show()

In [ ]:
fig=px.histogram(df,'Age')
fig.add_vline(x=df['Age'].mean(), line_width=2, line_dash="dash", line_color="black")

fig.show()


In [ ]:
fig = px.pie(df, names = "Gender",
             title = "<b>Counts in Gender</b>",
             hole = 0.5, template = "plotly_dark")

fig.update_traces(textposition='inside',
                  textinfo='percent+label',
                  marker=dict(line=dict(color='#000000', width = 1.5)))


fig.show()

In [ ]:
no_internship=df[df['Internships']==0]
no_internship.shape

In [ ]:
fig=px.histogram(no_internship,x='PlacedOrNot',color='PlacedOrNot')
fig.update_layout(bargap=0.3)
fig.show()


In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df['Stream']=le.fit_transform(df['Stream'])
df['Gender']=le.fit_transform(df['Gender'])

In [ ]:
df.head(n=20)

In [ ]:
x=df.iloc[:,:-1]
y=df.iloc[:,-1]
print(x.shape,y.shape)

In [ ]:
print(type(x))

In [ ]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x=sc.fit_transform(x)

In [ ]:
x

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=1,stratify=y)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
x_train

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dt=DecisionTreeClassifier()
dt.fit(x_train,y_train)

In [ ]:
dt.score(x_train,y_train)

In [ ]:
dt.score(x_test,y_test)

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
params={
    
     'max_depth': [2, 3, 5, 10, 20],
     'min_samples_leaf': [5, 10, 20, 50, 100],
     'criterion': ["gini", "entropy"]
}

In [ ]:
grid_search = GridSearchCV(estimator=dt, 
                           param_grid=params, 
                           cv=4, n_jobs=-1, verbose=1, scoring = "accuracy")

In [ ]:
grid_search.fit(x_train,y_train)

In [ ]:
grid_search.best_estimator_

In [ ]:
d_new=DecisionTreeClassifier(max_depth=20, min_samples_leaf=10)

In [ ]:
d_new.fit(x_train,y_train)

In [ ]:
d_new.score(x_train,y_train)

In [ ]:
d_new.score(x_test,y_test)

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier

base_models=[('dt',DecisionTreeClassifier(max_depth=20, min_samples_leaf=10,criterion='gini')),('RF',RandomForestClassifier(n_estimators=800)),('lr',LogisticRegressionCV())]
meta_model = LogisticRegressionCV()

In [ ]:
stacking_model = StackingClassifier(estimators=base_models, final_estimator=meta_model, passthrough=True, cv=3)


In [ ]:
stacking_model.fit(x_train, y_train)

In [ ]:
stacking_model.score(x_test,y_test)